In [28]:
import os
import argparse
import io
import glob
import hashlib
import tensorflow as tf
from pathlib import Path
from PIL import Image
import subprocess

In [29]:
def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))


In [30]:
def get_tf_record(image_path, label_path, class_names):
    #load image
    with tf.gfile.GFile(image_path, 'rb') as fid:
        encoded_img = fid.read()

    encoded_img_io = io.BytesIO(encoded_img)
    image = Image.open(encoded_img_io)
    key = hashlib.sha256(encoded_img).hexdigest()

    width, height = image.size

    #read annotation
    with open(label_path, 'r') as li:
        annotations = li.readlines()

    #convert annotations to tensorflow format
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes = []
    classes_text = []
    truncated = []
    poses = []
    difficult_obj = []
    for a in annotations:
        c_id, x, y, w, h = a.strip().split(' ')
        c_id=int(c_id)
        x=float(x)
        y=float(y)
        w=float(w)
        h=float(h)

        xmin.append(float(x - (w / 2)))
        ymin.append(float(y - (h / 2)))
        xmax.append(float(x + (w / 2)))
        ymax.append(float(y + (h / 2)))
        #class 0 is for background?
        classes.append(c_id+1)
        classes_text.append(class_names[c_id].encode('utf8'))
        #????????????????????????????????????
        truncated.append(0)
        poses.append(''.encode('utf8'))
        difficult_obj.append(int(False))

    #create tfrecords
    example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(image_path.encode('utf8')),
        'image/source_id': bytes_feature(image_path.encode('utf8')),
        'image/key/sha256': bytes_feature(key.encode('utf8')),
        'image/encoded': bytes_feature(encoded_img),
        'image/format': bytes_feature(image_path[-3:].encode('utf8')),
        'image/object/bbox/xmin': float_list_feature(xmin),
        'image/object/bbox/xmax': float_list_feature(xmax),
        'image/object/bbox/ymin': float_list_feature(ymin),
        'image/object/bbox/ymax': float_list_feature(ymax),
        'image/object/class/text': bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(classes),
        'image/object/difficult': int64_list_feature(difficult_obj),
        'image/object/truncated': int64_list_feature(truncated),
        'image/object/view': bytes_list_feature(poses),
    }))

    return example

In [31]:
def convert_yolo_tf(train_file,tf_record_out,label_name_out,class_to_name):
    try:
        file_list=[]
        with open(train_file) as f_in:
            file_list=f_in.readlines()
        
        class_names=[]
        with open(class_to_name) as f_in:
            class_names=f_in.readlines()
        class_names=[c.strip() for c in class_names if len(c)>0]
    except Exception as e:
        print('Caught Exception: {}'.format(e))
        print('Shutting Down')
        exit()
    
    label_list = [f.strip().replace('images','labels')[:-4]+'.txt' for f in file_list]
    

    #create tensorflow writer to write the final tfrecord
    writer = tf.python_io.TFRecordWriter(tf_record_out)

    for idx,(f,l) in enumerate(zip(file_list,label_list)):
        #get a single tfrecord describing the image and annotations
        example = get_tf_record(f.strip(), l, class_names)

        #write tfrecord
        writer.write(example.SerializeToString())

        print('{}/{}'.format(idx, len(file_list)),end='\r')

    print('\nTFRecord saved, creating label_name.pbtxt')
    with open(label_name_out, 'w+') as f_out:
        proto_string="\nitem{{\n\tid: {}\n\tname: '{}' \n }}\n"
        for i,c in enumerate(class_names):
            f_out.write(proto_string.format(i+1,c))
    
    print('Conversion Done')

In [32]:
DATASET_PATH = "/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/dataset"
TF_FILE_PATH = "/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/tf_data.tfrecord"

img_file_list = glob.glob(os.path.join(DATASET_PATH,"*.jpg"), recursive=True)

In [33]:
class_names = {0:"door",
              1:"handle",
              2:"cabinet door",
              3:"refrigerator door"}

In [35]:
writer =  tf.io.TFRecordWriter(TF_FILE_PATH)

for i,img in enumerate(img_file_list) :
    name = Path(img).stem
    img_path = os.path.join(DATASET_PATH , name + ".jpg")
    annotation_path = os.path.join(DATASET_PATH ,name  + ".txt")
    
    tf_record = get_tf_record(img_path,annotation_path,class_names)
    #print(tf_record.SerializeToString())
    writer.write(tf_record.SerializeToString())
    
    print(i," ",name," written to tf record.")


0   0e3381d79d5b1ea1  written to tf record.
1   0d5c075e91b9309e  written to tf record.
2   be1ae75c87fb45d1  written to tf record.
3   0e87814cbe9dd3c6  written to tf record.
4   0dbbd004c90cddf6  written to tf record.
5   0dbae2ad6cb64fbe  written to tf record.
6   520492e209ebc73a  written to tf record.
7   082d92e88891536a  written to tf record.
8   e75c980c218966f0  written to tf record.
9   0a68ceae0ff06341  written to tf record.
10   8cf0d90fadf83514  written to tf record.
11   d2c8e22257059647  written to tf record.
12   16f7251ee8768470  written to tf record.
13   satin-nickel-cup-pulls-satin-nickle-drawer-pulls-satin-nickel-drawer-hardware-knobs-amp-bin-pulls-satin-nickel-brushed-nickel-satin-nickle-drawer-pulls-brushed-nickel-3-inch-satin-nickel-cup-pulls  written to tf record.
14   8c0ce81d2f584ae7  written to tf record.
15   0c04f73a2916eece  written to tf record.
16   4d550dc12b4d59f8  written to tf record.
17   c76a6c85cb54c105  written to tf record.
18   0118064eedc2b2c

In [1]:
PIPELINE_CONFIG_PATH = "/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/Model/ssd_mobilenet_v1_ppn_shared_box_predictor_300x300_coco14_sync.config"
MODEL_DIR="/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/Model/"
NUM_TRAIN_STEPS=50000
SAMPLE_1_OF_N_EVAL_EXAMPLES=1

In [2]:
python object_detection/model_main.py \
    --pipeline_config_path=${PIPELINE_CONFIG_PATH} \
    --model_dir=${MODEL_DIR} \
    --num_train_steps=${NUM_TRAIN_STEPS} \
    --sample_1_of_n_eval_examples=$SAMPLE_1_OF_N_EVAL_EXAMPLES \
    --alsologtostderr

SyntaxError: invalid syntax (<ipython-input-2-b5502f1927a0>, line 1)

In [3]:
cmd = ['python','/home/nj/HBRS/RnD/Github/models/research/object_detection/model_main.py',
        '--pipeline_config_path='+PIPELINE_CONFIG_PATH,
        '--model_dir='+MODEL_DIR,
        '--num_train_steps='+str(NUM_TRAIN_STEPS),
        '--sample_1_of_n_eval_examples='+str(SAMPLE_1_OF_N_EVAL_EXAMPLES),
        '--alsologtostderr']
print(' '.join(cmd))

python /home/nj/HBRS/RnD/Github/models/research/object_detection/model_main.py --pipeline_config_path=/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/Model/ssd_mobilenet_v1_ppn_shared_box_predictor_300x300_coco14_sync.config --model_dir=/home/nj/HBRS/RnD/Dataset/DoorDetect-Dataset/Model/ --num_train_steps=50000 --sample_1_of_n_eval_examples=1 --alsologtostderr


In [18]:
p = subprocess.Popen(cmd, stdout=subprocess.PIPE, bufsize=1)
for line in iter(p.stdout.readline, b''):
    print(line),
p.stdout.close()
p.wait()

2

## References
1. https://towardsdatascience.com/working-with-tfrecords-and-tf-train-example-36d111b3ff4d
2. https://github.com/AlessioTonioni/tf-objdetector/blob/master/yolo_tf_converter.py

In [ ]:
python "/home/nj/HBRS/R&D/Github/models/research/object_detection/model_main.py" --pipeline_config_path="/home/nj/HBRS/R&D/Github/models/research/object_detection/samples/configs/ssd_mobilenet_v1_quantized_300x300_coco14_sync.config" --model_dir="/home/nj/HBRS/R&D/Dataset/DoorDetect-Dataset/Model/" --num_train_steps=50000 --sample_1_of_n_eval_examples=1 --alsologtostderr